# ¿Es este el verano más caluroso de los últimos años?

Este verano del 2023 viene siendo muy intenso para la mayoría de gente. He escuchado y leído por las redes sociales a muchas personas decir que este verano lo sienten como el más caluroso de los últimos años.

Sin embargo, me percaté de que cada año que pasa, la gente piensa esto mismo nuevamente. Por lo que me propuse buscar los datos históricos de las temperaturas medias de Buenos Aires y hacer un análisis para comprobar si realmente este año ha sido hasta ahora el verano con las mayores temperaturas medias de los ultimos años.  

Pude encontrar un dataset considerablemente completo, con un historial de temperaturas registradas provincia por provincia de Argentina en una web del gobierno en [este link](https://datos.gob.ar/dataset/smn-estadisticas-climaticas-normales). Pero desafortunadamente el registro llegaba hasta el año 2010 por lo que no me fue de gran utilidad.

Luego encontré en una web de la Ciudad este dataset con las temperaturas máximas y mínimas medias de cada mes entre 1991 y 2023, el más actualizado hasta la actualidad. Por lo tanto, ante la imposibilidad de encontrar información suficiente sobre las temperaturas históricas de la provincia de Buenos Aires, me decidí por realizar el proyecto con el registro de CABA. [Fuente aquí](https://www.estadisticaciudad.gob.ar/eyc/?p=27702).

## Data parsing

Uso la biblioteca `xlrd` para leer archivos de Excel

In [ ]:
!pip install xlrd

In [6]:
import xlrd
import json

workbook = xlrd.open_workbook("MA_AX01.xls")
sheet = workbook.sheet_by_index(0) # obtengo la primer hoja

In [7]:
dataClima = {}
            
for año in range (1992, 2024):
    dataClima[año] = {"Dic": [0,0], "Ene": [0,0], "Feb": [0,0]}

A partir del año 2018 comenzaron a incluir la temperatura media de cada mes, pero no la tomaré. Primero lleno el datClima con los años 1991 - 2017, y  luego del 2018-2023. Las columnas con las temperaturas medias van de 2 a 54 y las filas de cada mes son 4,5 y 15

In [8]:
columna = 3

for año in range (1992,2018):
    dataClima[año]["Dic"] = [sheet.cell_value(14,columna-2), sheet.cell_value(14,columna-1)]
    dataClima[año]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
    dataClima[año]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]
        
    columna += 2

# Arreglo los indices de columna por el desfase a partir del 2018
    
columna += 1
dataClima[2018]["Dic"] = [sheet.cell_value(14,columna-3), sheet.cell_value(14,columna-2)]
dataClima[2018]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
dataClima[2018]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]


columna += 3
        
for año in range (2019, 2024):
    dataClima[año]["Dic"] = [sheet.cell_value(14,columna-3), sheet.cell_value(14,columna-2)]
    dataClima[año]["Ene"] = [sheet.cell_value(3,columna), sheet.cell_value(3,columna+1)]
    dataClima[año]["Feb"] = [sheet.cell_value(4,columna), sheet.cell_value(4,columna+1)]
    
    columna += 3
    
# print (json.dumps(dataClima, indent=4))

## Data analysis

### Temps extremas medias

Ahora obtendré la media de las temperaturas máximas y mínimas medias de cada verano.

In [13]:
tempsMediasDeVeranos = []

def calcTempsMedia (añoNum, añoDict): # recibe cada diccionario dentro del diccionario dataClima
    mediaMax = (añoDict["Dic"][0] + añoDict["Ene"][0] + añoDict["Feb"][0])/3
    mediaMin = (añoDict["Dic"][1] + añoDict["Ene"][1] + añoDict["Feb"][1])/3
    
    return (añoNum,(round(mediaMax,1), round(mediaMin,1)))

for año in range (1992, 2024):
    duplaAñoMedias = calcTempsMedia(año, dataClima[año])
    tempsMediasDeVeranos.append(duplaAñoMedias)

Las ordeno descendentemente para ver en el inicio de cada lista los años que me interesan.

In [19]:
mediasMaximasOrdenadas = sorted(tempsMediasDeVeranos, key = lambda x:x[1][0], reverse=True)
mediasMinimasOrdenadas = sorted(tempsMediasDeVeranos, key = lambda x:x[1][1], reverse=True)

print(mediasMaximasOrdenadas)
print("")
print(mediasMinimasOrdenadas)

[(2023, (31.2, 20.4)), (2016, (30.7, 20.7)), (2018, (30.7, 20.1)), (2017, (30.4, 20.4)), (2014, (30.2, 20.1)), (2009, (30.1, 19.8)), (2012, (30.0, 19.4)), (2008, (29.9, 19.8)), (2011, (29.9, 20.0)), (1997, (29.8, 20.1)), (2000, (29.8, 19.9)), (2013, (29.8, 19.5)), (1996, (29.7, 19.4)), (2007, (29.7, 19.8)), (2020, (29.7, 19.2)), (2001, (29.6, 19.6)), (1995, (29.5, 19.5)), (2022, (29.5, 20.1)), (2005, (29.3, 19.6)), (2015, (29.3, 19.0)), (2021, (29.3, 19.1)), (2003, (28.8, 18.9)), (2010, (28.8, 19.7)), (2006, (28.7, 18.6)), (2002, (28.6, 18.6)), (2004, (28.6, 18.2)), (2019, (28.6, 19.2)), (1993, (28.5, 19.6)), (1992, (28.4, 19.4)), (1999, (27.9, 18.2)), (1994, (27.7, 18.7)), (1998, (26.7, 17.8))]

[(2016, (30.7, 20.7)), (2017, (30.4, 20.4)), (2023, (31.2, 20.4)), (1997, (29.8, 20.1)), (2014, (30.2, 20.1)), (2018, (30.7, 20.1)), (2022, (29.5, 20.1)), (2011, (29.9, 20.0)), (2000, (29.8, 19.9)), (2007, (29.7, 19.8)), (2008, (29.9, 19.8)), (2009, (30.1, 19.8)), (2010, (28.8, 19.7)), (1993, 

### Variación porcentual reciente

Me interesa también saber la variación porcentual de las medias del año 2023 respecto a años anteriores, para saber si realmente hay un porcentaje distinguible de aumento de temperaturas, lo que puede llevar a la gente a notar este cambio. Tomaré las medias de los ultimos 3 y 5 veranos pasados, para quedarme con los años que una persona puede recordar con mayor facilidad segun el calor "sufrido".

In [35]:
mediasMaximasUlt3 = 0;
mediasMaximasUlt5 = 0;

for i in range(-4, -1):
    mediasMaximasUlt3 += tempsMediasDeVeranos[i][1][0]
    
for i in range(-6,-1):
    mediasMaximasUlt5 += tempsMediasDeVeranos[i][1][0]
    
mediasMaximasUlt3 = round(mediasMaximasUlt3/3, 1)
mediasMaximasUlt5 = round(mediasMaximasUlt5/5, 1)

mediaMax2023 = tempsMediasDeVeranos[-1][1][0]

## Conclusión

Debajo, el top 5 de años con medias de temperaturas máximas más altas registradas en los últimos 31 años:

In [21]:
for i in range(5):
    print(str(mediasMaximasOrdenadas[i][0]) + ": con media de maximas " + str(mediasMaximasOrdenadas[i][1][0]))

De mayor a menor, los 5 años con las medias de temperaturas maximas más altas y sus respectivas medias, de los ultimos 31 años: 
2023: con media de maximas 31.2
2016: con media de maximas 30.7
2018: con media de maximas 30.7
2017: con media de maximas 30.4
2014: con media de maximas 30.2


Debajo, el top 5 de años con medias de temperaturas mínimas más altas registradas en los últimos 31 años:

In [23]:
for i in range(5):
    print(str(mediasMinimasOrdenadas[i][0]) + ": con media de mínimas " + str(mediasMinimasOrdenadas[i][1][1]))

2016: con media de mínimas 20.7
2017: con media de mínimas 20.4
2023: con media de mínimas 20.4
1997: con media de mínimas 20.1
2014: con media de mínimas 20.1


¿Cuál ha sido la variación porcentual respecto a los últimos años de la media de temps máximas? Teniendo en cuenta que la del 2023 fue la mayor de todas, podemos simplemente calcular qué porcentaje ha aumetnado

In [36]:
print ("La media de temperaturas maximas del 2023 fue " + str(round((mediaMax2023*100/mediasMaximasUlt3)-100, 1)) + "% mayor que la de los últimos 3 años.")
print ("La media de temperaturas maximas del 2023 fue " + str(round((mediaMax2023*100/mediasMaximasUlt5)-100, 1)) + "% mayor que la de los últimos 5 años.")

La media de temperaturas maximas del 2023 fue 5.8% mayor que la de los últimos 3 años.
La media de temperaturas maximas del 2023 fue 5.4% mayor que la de los últimos 5 años.


Como se puede apreciar, las temperaturas máximas registradas en el verano 2022-2023 (es decir, Diciembre del 2022, y Enero y Febrero del 2023) dieron el promedio más alto de las últimas tres décadas, segun los datos registrados por el Servicio Meteorológico Nacional y el Observatorio Buenos Aires. Y además, si tomamos las temperaturas mínimas de cada verano, el verano 2022-2023 tuvo el segundo (o tercero) promedio más alto de las últimas tres décadas.  

Por lo tanto, se podría decir que, efectivamente, este verano ha sido el más caluroso de las últimas tres décadas, tomando en cuenta el promedio de los pícos de temperaturas máximas registradas en los meses de Dic-Ene-Feb. Y asimismo, podría decirse que este verano ha sido el seguno "menos frío" de las últimas tres décadas, tomando en cuenta los promedios de temperaturas mínimas. Además, volviendo al asunto por el que me propuse iniciar este proyecto, el aumento porcentual de temperaturas 